In [1]:
import numpy as np
import skimage.io 
import matplotlib.pyplot as plt 
import os, sys
import pickle
import pandas as pd 
from skimage.registration import phase_cross_correlation
import cv2 
import itertools
from pathlib import Path

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# Import path 
module_path = str(Path.cwd().parents[0])
if module_path not in sys.path:
    sys.path.append(module_path)

from config import *

In [6]:
csv_file = data_meta / 'all_info.csv'
csv_exist = csv_file.is_file()

# Create directory if not existL
try:
    data_processed.mkdir(parents=True, exist_ok=False)
except FileExistsError:
    print("Processed image folder is already there")
else:
    print("Processed image folder was created")
    
if csv_exist:
    print('Info csv file already exists')
else:
    print('No Info csv file')

Processed image folder is already there
No Info csv file


# Get folder information

Read information to pandas dataframe of the image folder

In [7]:
channel2marker = {
    '2seg':
    {
        'CH1' : 'DAPI', 
        'CH4' : 'WGA',
        'CH3' : 'Phalloidin'
    },
    'Concanavadin A':
    {
        'CH1' : 'DAPI', 
        'CH2' : 'Concanavadin A'
    },
    'CycD1':
    {
        'CH1' : 'DAPI', 
        'CH4' : 'Cyclin D1',
        'CH3' : 'APC'
    }
    ,'CycE':
    {
        'CH1' : 'DAPI', 
        'CH4' : 'WNT-1',
        'CH3' : 'EMMPRIN',
        'CH2' : 'Cyclin E'
    }
    ,'DKK1':
    {
        'CH1' : 'DAPI', 
        'CH4' : 'DKK1',
        'CH2' : 'Non-phospho-B-catenin'
    }
}

def get_info(img_folder, name_dict = channel2marker, save=False):
    '''Function returns the info from folder containing multi-cycle staigning on cell
    
    Args: 
        img_folder (str) : imgage folder path to get information
        name_dict (dict) : three level dictionnary mapping cycle -> channel -> marker name
        
    Returns:
        pandas dataframe with information
    '''
    conditions = []
    cycles = []
    z_stacks = []
    channels = []
    images_path = []
    markers = []

    # Loop through image folder
    for (dirpath, dirnames, filenames) in os.walk(img_folder):
        for name in sorted(filenames):
            if 'aftBleach' not in dirpath and 'Fw' in dirpath:            
                if 'tif' in name and 'Overlay' not in name and 'FF_' not in name:
                    # Get information from image name 
                    condition = dirpath[-3:]
                    z_stack = '_'.join(name.split('_')[2:3])
                    channel = name.split('_')[-1].split('.')[0]
                    cycle = dirpath.split('\\')[-1].split('_')[1]

                    if z_stack == "":
                        continue
                        
                    markers.append(name_dict[cycle][channel])

                    cycles.append(cycle)
                    conditions.append(condition)
                    z_stacks.append(int(z_stack[1:]))
                    channels.append(channel)
                    images_path.append(os.path.join(dirpath,name))

    info = {'Cycle':cycles, 'Condition':conditions, 'Z_stack': z_stacks, 'Channel': channels, 'Marker': markers,'Path': images_path}

    df = pd.DataFrame(info)
    
    if save == True:
        df.to_csv('info.csv', index=False)
        
    df.Z_stack = pd.to_numeric(df.Z_stack)
    
    df['Path_corrected'] = ''
    
    return df

In [8]:
if not csv_exist:
    df = get_info(data_raw)
    print('Created pandas dataframe')
else:
    df = pd.read_csv(csv_file)
    print('Imported pandas dataframe')

Created pandas dataframe


# Image background subtraction

Image shadding correction using background subtration of gaussian blurr

In [12]:
def extract_condition_channel(df, cycle, condition, channel):
    '''Function returns dataframe of specified channel image based on condition from input dataframe
    
    Args: 
        df (pd DataFrame) : info dataframe for all images 
        condition (str) : condition name
        
    Returns:
        pandas dataframe with information
    '''
    return df[(df['Condition'] == condition) & (df['Channel'] == channel) & (df['Cycle'] == cycle)]

def background_correction(df, filtersize = 257, sigma = 128, folder = data_processed, save=False, show=False):
    '''Function to perform background substraction for image using gaussian blurr of original image 
    
    Args: 
        df (pd DataFrame) : info dataframe for all images 
        filtersize (int) : filter size of gaussian kernel
        sigma (int) : sigma of guassian blurr
        folder (str) : folder to save corrected images
        save (bool) : bool to save the image
        show (bool) : bool to show corrected image and gaussian blur of original image
        
    Returns:
        None
    '''
    # Loop through the rows of the dataframe
    for row in df.itertuples():
        # Read image
        img = skimage.io.imread(row.Path)
        
        # Define saving filename for corrected image
        filename = '_'.join([row.Condition, str(row.Z_stack), row.Cycle, row.Marker])
        path = os.path.join(folder,filename+'.tiff')
        
        # If marker is Phalloidin no correction needed for better segmentation:
        if row.Marker == 'Phalloidin':
            if save:
                cv2.imwrite(path, img) 
                df.at[row.Index,'Path_corrected'] = path
            continue
        
        # Background substraction using gaussian blur channel
        gaussianImg = cv2.GaussianBlur(img, (filtersize, filtersize), sigma)
        img_corrected = cv2.subtract(img,gaussianImg)
        
        # Save image
        if save:
            cv2.imwrite(path, img_corrected) 
            df.at[row.Index,'Path_corrected'] = path
        
        # Show correction
        if show:
            fig = plt.figure(figsize=(25, 7))
            ax1 = plt.subplot(1, 3, 1)
            ax2 = plt.subplot(1, 3, 2)
            ax3 = plt.subplot(1, 3, 3)

            ax1.imshow(img, alpha=1)
            ax1.set_axis_off()
            ax1.set_title('Original Image')

            ax2.imshow(img_corrected, alpha=1)
            ax2.set_axis_off()
            ax2.set_title('Corrected image')

            ax3.imshow(gaussianImg, cmap='gray', alpha=1)
            ax3.set_axis_off()
            ax3.set_title('Background image')
            plt.show()

In [13]:
if not csv_file.is_file():
    background_correction(df, save=True)
    df.to_csv(csv_file, index=False)
    print('Created images with background correction')

Created images with background correction


# Image registration

Image registration using the DAPI channel across images

In [14]:
def get_shift_within_cycle(df):
    '''Function to get shift within each cycle of the DAPI channel 
    
    Args: 
        df (pd DataFrame) : info dataframe for images 
        
    Returns:
        print shift values between each level
    '''
    # Get elements of DAPI channel
    df = df[df['Marker'] == 'DAPI']
    
    # Loop through condition
    for condition in df.Condition.unique():
        for cycle in df.Cycle.unique():
            # Read the cycle info and z level of 1 for reference dapi channel
            temp = df[(df.Cycle == cycle) & (df.Condition == condition)]
            reference = temp[temp.Z_stack == 1]
            reference_dapi = skimage.io.imread(reference.Path_corrected.item())

            # number of z level in this cycle
            n_channel = temp.Z_stack.max()+1
            
            # Get shift between z level 
            for i in range(2, n_channel):
                img_dapi = skimage.io.imread(temp[temp.Z_stack == i].Path_corrected.item())
                shift, error, diffphase = phase_cross_correlation(reference_dapi, img_dapi)
                print(f"For {cycle} detected subpixel offset ({shift[0]}, {shift[1]}) between level 1 and level {i} stack for {condition}")
            
def get_shift_between_cycle(df):
    '''Function to get shift within each cycle for the DAPI channel
    
    Args: 
        df (pd DataFrame) : info dataframe for images in dapi channel accross cycle
        
    Returns:
        shift dictionnary
    '''
    # Get elements of DAPI channel
    df = df[df['Marker'] == 'DAPI']
    
    # Get shift between cycle
    cycles = df.Cycle.unique()
    conditions = df.Condition.unique()
    shift_dict = {}
    for condition in conditions:
        # Get subset of specific condition
        df_subset = df[df.Condition == condition]
        
        # Get reference cycle
        temp = df_subset[df_subset.Cycle == cycles[0]]  
        reference = temp[temp.Z_stack == 1]
        reference_dapi = skimage.io.imread(reference.Path_corrected.item())
    
        # Define shift dictionnary
        shift_dict[condition] = {}

        # Max shift accross all cycle:
        max_shift_x = 0
        min_shift_x = 0
        max_shift_y = 0
        min_shift_y = 0

        for cycle in cycles[1:]:
            temp = df_subset[df.Cycle == cycle]    
            img_dapi = skimage.io.imread(temp[temp.Z_stack == 1].Path_corrected.item())

            # Get image shift y and x and save to shift_dict
            shift, _, _ = phase_cross_correlation(reference_dapi, img_dapi) # Shift vector required to register moving images with reference images. Axis orderingis constitent with Y,X
            print(f"For {cycle} detected subpixel offset ({shift[0]}, {shift[1]}) compared to {cycles[0]} for {condition}")
            shift_y, shift_x = shift[0], shift[1]
            shift_dict[condition][cycle] = {'shift_x':shift_x, 'shift_y':shift_y}

            # Update max shift
            max_shift_x = shift_x if shift_x > max_shift_x else max_shift_x
            min_shift_x = shift_x if shift_x < min_shift_x else min_shift_x
            max_shift_y = shift_y if shift_y > max_shift_y else max_shift_y
            min_shift_y = shift_y if shift_y < min_shift_y else min_shift_y

        max_shift_x = int(max_shift_x)
        min_shift_x = int(min_shift_x)
        max_shift_y = int(max_shift_y)
        min_shift_y = int(min_shift_y)

        shift_dict[condition]['max'] = {'max_shift_x':max_shift_x, 'min_shift_x':min_shift_x, 'max_shift_y':max_shift_y, 'min_shift_y':min_shift_y}

    return shift_dict

def shift_crop(df, shift_dict, save = True):
    '''Function to shift and crop image based on shift_dict of pixels
    
    Args: 
        df (pd DataFrame) : info dataframe for images 
        shift_dict (dict) : shfit dictionnary between each cycle by condition
        save (bool) : bool to save the images
        
    Returns:
        None
    '''
    # Iterate over condition
    for condition in df.Condition.unique():
        shift_dict_subset = shift_dict[condition]
        
        # Get max shift values
        max_shift_x = shift_dict_subset['max']['max_shift_x']
        min_shift_x = shift_dict_subset['max']['min_shift_x']
        max_shift_y = shift_dict_subset['max']['max_shift_y']
        min_shift_y = shift_dict_subset['max']['min_shift_y']
    
        # Get dataframe subset corresponding to condition 
        df_subset = df[df.Condition == condition]
        
        # iterate over rows:
        for row in df_subset.itertuples():
            img = skimage.io.imread(row.Path_corrected)
            condition = row.Condition

            # If not in shift dict then reference
            if row.Cycle not in shift_dict_subset.keys():
                # Crop image
                res_cropped = img[max_shift_y: min_shift_y, max_shift_x:min_shift_x]
                
            else:
                # Aplly shift 
                shift_x, shift_y = shift_dict_subset[row.Cycle]['shift_x'], shift_dict_subset[row.Cycle]['shift_y']
                rows,cols = img.shape
                M = np.float32([[1,0,shift_x],[0,1,shift_y]])
                res = cv2.warpAffine(img, M, (cols, rows)) 

                # Crop image
                res_cropped = res[max_shift_y: min_shift_y, max_shift_x:min_shift_x]
                
            if save: 
                cv2.imwrite(row.Path_corrected, res_cropped)

In [15]:
if not csv_exist:
    get_shift_within_cycle(df)

For 2seg detected subpixel offset (0.0, 0.0) between level 1 and level 2 stack for Fw1
For 2seg detected subpixel offset (0.0, 0.0) between level 1 and level 3 stack for Fw1
For 2seg detected subpixel offset (0.0, 0.0) between level 1 and level 4 stack for Fw1
For Concanavadin A detected subpixel offset (0.0, 0.0) between level 1 and level 2 stack for Fw1
For Concanavadin A detected subpixel offset (0.0, 0.0) between level 1 and level 3 stack for Fw1
For Concanavadin A detected subpixel offset (0.0, 0.0) between level 1 and level 4 stack for Fw1
For Concanavadin A detected subpixel offset (-2.0, 1.0) between level 1 and level 5 stack for Fw1
For Concanavadin A detected subpixel offset (-2.0, 1.0) between level 1 and level 6 stack for Fw1
For CycD1 detected subpixel offset (0.0, 0.0) between level 1 and level 2 stack for Fw1
For CycD1 detected subpixel offset (0.0, 0.0) between level 1 and level 3 stack for Fw1
For CycD1 detected subpixel offset (0.0, 0.0) between level 1 and level 4 st

In [16]:
if not csv_exist:
    shift_dict = get_shift_between_cycle(df)

For Concanavadin A detected subpixel offset (-24.0, 7.0) compared to 2seg for Fw1
For CycD1 detected subpixel offset (-18.0, -38.0) compared to 2seg for Fw1
For CycE detected subpixel offset (-12.0, -6.0) compared to 2seg for Fw1
For DKK1 detected subpixel offset (-15.0, 4.0) compared to 2seg for Fw1
For Concanavadin A detected subpixel offset (-14.0, -3.0) compared to 2seg for Fw2
For CycD1 detected subpixel offset (-8.0, -24.0) compared to 2seg for Fw2
For CycE detected subpixel offset (-10.0, -2.0) compared to 2seg for Fw2
For DKK1 detected subpixel offset (-7.0, -8.0) compared to 2seg for Fw2
For Concanavadin A detected subpixel offset (-26.0, -10.0) compared to 2seg for Fw3
For CycD1 detected subpixel offset (0.0, -10.0) compared to 2seg for Fw3
For CycE detected subpixel offset (-10.0, -8.0) compared to 2seg for Fw3
For DKK1 detected subpixel offset (-15.0, -12.0) compared to 2seg for Fw3


In [17]:
if not csv_exist:
    shift_crop(df, shift_dict, save=True)

# Combine z_stack 

In [20]:
def get_maximum_projection(df, img_folder, save=False):
    # Read condition and cycles name
    conditions = df.Condition.unique()
    cycles = df.Cycle.unique()

    # Loop through condition and cycle pair
    for ele in itertools.product(conditions,cycles):
        condition = ele[0]
        cycle = ele[1]

        # Extract dataset  
        df_subset = df[(df.Cycle == cycle) & (df.Condition == condition)] 
        for marker in df_subset.Marker.unique():
            df_channel = df_subset[df_subset.Marker == marker]
            imgs = None
            for row in df_channel.itertuples():
                img = skimage.io.imread(row.Path_corrected)
                imgs = np.concatenate((imgs, img[np.newaxis,:,:]), axis = 0) if not (imgs is None) else img[np.newaxis,:,:]

            # Save image
            img_combined = np.amax(imgs, axis=0, keepdims=True)[0]
            filename = '_'.join([row.Condition, 'Combined', row.Cycle, row.Marker])
            path = os.path.join(img_folder, filename+'.tiff')
            cv2.imwrite(path, img_combined)

            # Append dataframe
            new_row = {
                'Cycle': row.Cycle, 
                'Condition': row.Condition,
                'Z_stack': 'Combined',
                'Channel': row.Channel,
                'Marker': row.Marker,
                'Path': '',
                'Path_corrected': path
            }


            df = df.append(new_row, ignore_index = True)

    if save:
        df.to_csv(csv_file, index=False)  

In [21]:
get_maximum_projection(df, data_processed, save=True)

# Detect dot in images

In [12]:
# from skimage.feature import blob_log

# blobs_log = blob_log(img_corrected, max_sigma=30, num_sigma=10, threshold=.05)

In [13]:
# fig, ax = plt.subplots(figsize=(15, 15))
# ax.imshow(img_corrected)
# for blob in blobs_log:
#     y, x, r =blob
#     c = plt.Circle((x, y), r, color='red', linewidth=2, fill=False)
#     ax.add_patch(c)
#     ax.set_axis_off()
    
# plt.tight_layout()
# plt.show()